In [109]:
import pandas as pd
import model_afolu as ma
import model_circular_economy as mc
import numpy as np
import os, os.path
import data_structures as ds
import setup_analysis as sa
import support_functions as sf
import importlib
importlib.reload(ds)
importlib.reload(sa)


#import data_structures as ds
#importlib.reload(ds)

/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:462: UserWarning: Invalid subsector attribute 'key_varreqs_all'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector'
  modvars.sort()
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:462: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector'
  modvars.sort()


<module 'setup_analysis' from '/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/setup_analysis.py'>

In [34]:
#model_ce = mc.CircularEconomy(sa.model_attributes)
model_afolu = ma.AFOLU(sa.model_attributes)


In [39]:

sa.model_attributes.get_variables_by_sector("Socioeconomic", "both")

['area_country_ha',
 'frac_eating_red_meat',
 'gdp_mmm_usd',
 'occ_rate',
 'population',
 'population_rural',
 'population_urban',
 'va_commercial_mmm_usd',
 'va_industrial_mmm_usd',
 'va_manufacturing_mmm_usd',
 'va_mining_mmm_usd']

In [110]:
# get the
tp = max(sa.model_attributes.get_time_periods()[0])
f0, f1 = (f"min_{tp}", f"max_{tp}")
df_template_afolu = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "tmp_afolu_vars_with_mcmc_new.csv"))
df_template_afolu.drop([f0, f1], axis = 1, inplace = True)

df_sampling_range_defaults = sa.model_attributes.build_default_sampling_range_df()

df_template_afolu = pd.merge(df_template_afolu, df_sampling_range_defaults, how = "left", on = ["variable"])
df_template_afolu = df_template_afolu.dropna(subset = ["max_35", "min_35"], how = "any")
df_template_afolu.drop(["src"], axis = 1, inplace = True)

df_template_afolu["time_series_id"] = np.zeros(len(df_template_afolu)).astype(int)
df_template_afolu["strategy_id"] = np.zeros(len(df_template_afolu)).astype(int)
df_template_afolu["uniform_scaling_q"].iloc[~df_template_afolu["uniform_scaling_q"].isna()] = df_template_afolu["uniform_scaling_q"].iloc[2]

# order
tp_max = max(sa.model_attributes.get_time_periods()[0])
fields_minmax = [f"min_{tp_max}", f"max_{tp_max}"]
fields_prep = [x for x in df_template_afolu.columns if not x.isnumeric() and (x not in fields_minmax)] + fields_minmax
fields_times = [x for x in df_template_afolu.columns if (x not in fields_prep)]

df_template_afolu = df_template_afolu[fields_prep + fields_times]
subsectors = [sa.model_attributes.dict_model_variable_to_subsector[sa.model_attributes.dict_variables_to_model_variables[x]] for x in df_template_afolu["variable"]]
df_template_afolu["subsector"] = subsectors

for sec in ["AFOLU", "Socioeconomic"]:

    vars_keep = sa.model_attributes.get_variables_by_sector(sec, "input")
    df_exp = df_template_afolu[df_template_afolu["variable"].isin(vars_keep)].copy().reset_index(drop = True).drop_duplicates().sort_values(by = ["subsector", "variable"])
    
    dict_out = {
        "strategy_id-0": df_exp, 
        "strategy_id-1": df_exp.iloc[0:0]
    }
    sf.dict_to_excel(sa.excel_template_path(sec, "arg", "demo", True), dict_out)



/Users/jsyme/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:462: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  modvars.sort()


In [113]:
##############################
#    BUILD WASTE TEMPLATE    #
##############################

sec = "Circular Economy"
l_vars = sa.model_attributes.get_variables_by_sector(sec, "input")
l_vars.sort()

df_base = df_exp.iloc[0:0].copy()
df_base["variable"] = l_vars
# add subsectors
subsectors = [sa.model_attributes.dict_model_variable_to_subsector[sa.model_attributes.dict_variables_to_model_variables[x]] for x in df_base["variable"]]
df_base["subsector"] = subsectors

df_base.drop(["min_35", "max_35"], axis = 1, inplace = True)
df_base = pd.merge(df_base, df_sampling_range_defaults)
df_base = df_base[df_template_afolu.columns]


dict_out = {
    "strategy_id-0": df_base, 
    "strategy_id-1": df_base.iloc[0:0]
}
sf.dict_to_excel(sa.excel_template_path(sec, "arg", "demo", True), dict_out)

    

In [115]:
sa.model_attributes.build_varlist("Solid Waste")

['frac_waso_composition_ind_chemical_industrial',
 'frac_waso_composition_ind_food',
 'frac_waso_composition_ind_glass',
 'frac_waso_composition_ind_metal',
 'frac_waso_composition_ind_nappies',
 'frac_waso_composition_ind_other',
 'frac_waso_composition_ind_paper',
 'frac_waso_composition_ind_plastic',
 'frac_waso_composition_ind_rubber_leather',
 'frac_waso_composition_ind_textiles',
 'frac_waso_composition_ind_wood',
 'frac_waso_composition_ind_yard',
 'physparam_waso_boc_chemical_industrial',
 'physparam_waso_boc_food',
 'physparam_waso_boc_glass',
 'physparam_waso_boc_metal',
 'physparam_waso_boc_nappies',
 'physparam_waso_boc_other',
 'physparam_waso_boc_paper',
 'physparam_waso_boc_plastic',
 'physparam_waso_boc_rubber_leather',
 'physparam_waso_boc_textiles',
 'physparam_waso_boc_wood',
 'physparam_waso_boc_yard',
 'frac_waso_composition_dom_chemical_industrial',
 'frac_waso_composition_dom_food',
 'frac_waso_composition_dom_glass',
 'frac_waso_composition_dom_metal',
 'frac_wa

In [151]:
v1 = np.array([12, 4, 10, 3, 3])
v2 = np.array([4, 10, 1, 11, 2])
#v2 = np.array([15, 4, 9, 3, 3])

np.arccos(np.dot(v1, v2)/((np.dot(v1, v1)*np.dot(v2, v2))**(0.5)))/np.pi


0.3228707368329673

In [ ]:
### importlib.reload(sf)
importlib.reload(ds)
importlib.reload(sa)
sa.excel_template_path("AFOLU", "arg", "demo", True)

In [360]:

?pd.DataFrame.to_excel

Signature:
pd.DataFrame.to_excel(
    self,
    excel_writer,
    sheet_name='Sheet1',
    na_rep='',
    float_format=None,
    columns=None,
    header=True,
    index=True,
    index_label=None,
    startrow=0,
    startcol=0,
    engine=None,
    merge_cells=True,
    encoding=None,
    inf_rep='inf',
    verbose=True,
    freeze_panes=None,
) -> None
Docstring:
Write object to an Excel sheet.

To write a single object to an Excel .xlsx file it is only necessary to
specify a target file name. To write to multiple sheets it is necessary to
create an `ExcelWriter` object with a target file name, and specify a sheet
in the file to write to.

Multiple sheets may be written to by specifying unique `sheet_name`.
With all data written to the file it is necessary to save the changes.
Note that creating an `ExcelWriter` object with a file name that already
exists will result in the contents of the existing file being erased.

Parameters
----------
excel_writer : str or ExcelWriter object
  

In [335]:
sa.model_attributes.get_sector_attribute("AFOLU", "abbreviation_sector")

'af'

In [132]:
df_template_afolu

,subsector,variable,time_series_id,strategy_id,variable_trajectory_group,variable_trajectory_group_trajectory_type,normalize_group,trajgroup_no_vary_q,uniform_scaling_q,0,...,28,29,30,31,32,33,34,35,max_35,min_35
0,Agriculture,ef_anaerobicdom_bevs_and_spices_kg_ch4_ha,0,0,NaN,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,Agriculture,ef_anaerobicdom_cereals_kg_ch4_ha,0,0,NaN,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,Agriculture,ef_anaerobicdom_coffee_kg_ch4_ha,0,0,NaN,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,Agriculture,ef_anaerobicdom_fibers_kg_ch4_ha,0,0,NaN,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,Agriculture,ef_anaerobicdom_fruits_kg_ch4_ha,0,0,NaN,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,Livestock,lvst_pop_initial_goats,0,0,NaN,NaN,NaN,NaN,NaN,25082.0,...,25082.0,25082.0,25082.0,25082.0,25082.0,25082.0,25082.0,25082.0,1.0,1.0
306,Livestock,lvst_pop_initial_horses,0,0,NaN,NaN,NaN,NaN,NaN,126033.0,...,126033.0,126033.0,126033.0,126033.0,126033.0,126033.0,126033.0,126033.0,1.0,1.0
307,Livestock,lvst_pop_initial_mules,0,0,NaN,NaN,NaN,NaN,NaN,5273.0,...,5273.0,5273.0,5273.0,5273.0,5273.0,5273.0,5273.0,5273.0,1.0,1.0
308,Livestock,lvst_pop_initial_pigs,0,0,NaN,NaN,NaN,NaN,NaN,462282.0,...,462282.0,462282.0,462282.0,462282.0,462282.0,462282.0,462282.0,462282.0,1.0,1.0


In [131]:
os.listdir(sa.dir_proj)

['ref',
 '.DS_Store',
 'LICENSE',
 'project_metadata.json',
 'python',
 'out',
 'pyproject.toml',
 'docs',
 'model_overview.md',
 'README.md',
 '.gitignore',
 'julia',
 'README.rst',
 '.git']